In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("excel_full_train.csv")
test = pd.read_csv("excel_test.csv")

,PassengerId,Survived,Pclass,Fare_1,Age_1,Family,Sex_1,IsAlone,Title_Mr,Title_Mrs,...,Ticket_C,Ticket_LINE,Ticket_FCC,Ticket_SWPP,Ticket_PPP,Ticket_SC,Ticket_SCAH,Ticket_SOPP,Ticket_FC,Ticket_SOTONO2
0,1,0,3,7.25,22.0,2,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = train.drop(['PassengerId','Survived'], axis = 1)

In [5]:
y = train['Survived']

### Feature Selection

In [6]:
from sklearn.feature_selection import SelectKBest, chi2

In [7]:
X.head(1)

,Pclass,Fare_1,Age_1,Family,Sex_1,IsAlone,Title_Mr,Title_Mrs,Title_Miss,Title_Master,...,Ticket_C,Ticket_LINE,Ticket_FCC,Ticket_SWPP,Ticket_PPP,Ticket_SC,Ticket_SCAH,Ticket_SOPP,Ticket_FC,Ticket_SOTONO2
0,3,7.25,22.0,2,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
sel_Feat = SelectKBest(chi2, k=30).fit(X, y)

In [9]:
sel_Bool = sel_Feat.get_support()

In [10]:
X_sel = X[X.columns[sel_Bool]]
X_sel.head()

,Pclass,Fare_1,Age_1,Sex_1,IsAlone,Title_Mr,Title_Mrs,Title_Miss,Title_Master,Embarked_S,...,Ticket_A4,Ticket_SOC,Ticket_WC,Ticket_SOTONOQ,Ticket_FCC,Ticket_SWPP,Ticket_SC,Ticket_SCAH,Ticket_SOPP,Ticket_SOTONO2
0,3,7.2500,22.0,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,71.2833,38.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,7.9250,26.0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,53.1000,35.0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,3,8.0500,35.0,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#### Use Test Train Split to divide into train and test
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sel, y, test_size=0.30, random_state=21)

In [12]:
from sklearn.ensemble import RandomForestClassifier

In [13]:
RF_Model = RandomForestClassifier(n_estimators = 100)

In [14]:
RF_Model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

### Grid Search

In [50]:
from sklearn.model_selection import GridSearchCV

In [51]:
#Using max_depth, criterion will suffice for DT Models, rest all will remain constant 
parameters = {'n_estimators' : (10,30,50,70,90,100)
              , 'criterion' : ('gini', 'entropy')
              , 'max_depth' : (3,5,7,9,10)
              , 'max_features' : ('auto', 'sqrt')
              , 'min_samples_split' : (2,4,6)
              #, 'min_weight_fraction_leaf' : (0.0,0.1,0.2,0.3)
             }

In [52]:
RF_grid  = GridSearchCV(RandomForestClassifier(n_jobs = -1, oob_score= False), param_grid = parameters, cv = 3, verbose = True)

In [53]:
RF_grid_model = RF_grid.fit(X_train, y_train)

Fitting 3 folds for each of 360 candidates, totalling 1080 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1080 out of 1080 | elapsed:  3.7min finished


In [54]:
RF_grid_model.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=7, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [55]:
RF_grid_model.best_score_

0.8282504012841091

In [65]:
#Build model with best estimates 
RF_Model = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=7, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [66]:
RF_Model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=7, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [67]:
RF_Model.score(X_train, y_train)

0.8828250401284109

In [68]:
best_feat = pd.DataFrame({'Features': X_train.columns,'Importance': RF_Model.feature_importances_})
best_feat.sort_values('Importance', ascending = False)

,Features,Importance
3,Sex_1,0.200631
5,Title_Mr,0.186577
1,Fare_1,0.124549
2,Age_1,0.106118
0,Pclass,0.085063
6,Title_Mrs,0.069460
7,Title_Miss,0.051211
11,Cabin_M,0.030428
9,Embarked_S,0.020199
4,IsAlone,0.019533


### Evaluation of Test

In [69]:
X_test_sel = X_test[X.columns[sel_Bool]]

In [70]:
X_test_sel.shape

(268, 30)

In [71]:
y_pred = RF_Model.predict(X_test_sel)

In [72]:
print(f'Test : {RF_Model.score(X_test, y_test):.3f}')
print(f'Train : {RF_Model.score(X_train, y_train):.3f}')

Test : 0.854
Train : 0.883


### Prediction of Test

In [73]:
sub_test = test.drop(['PassengerId'], axis = 1)

In [74]:
sub_test_sel = sub_test[X.columns[sel_Bool]]

In [75]:
sub_test_pred = RF_Model.predict(sub_test_sel).astype(int)

In [76]:
AllSub = pd.DataFrame({ 'PassengerId': test['PassengerId'],
                       'Survived' : sub_test_pred
    
})

AllSub.to_csv("Sel_Var_Video_Random_Forest_30.csv", index = False)

In [ ]:
#Kaggle LB Score - 0.78947